In [ ]:
# Monitors the "Doing" list

In [ ]:
from trello import TrelloListMonitor
from IFTTT import IFTTTNotifier
from habitica import HabiticaAPI
from stack_client import StackClient

In [ ]:
stack_client = StackClient()


In [ ]:
def desktop_notifier_callback(final_result, task_id, direction):
    # switch case where if task_id is trivial-doot, then stack_client.add_trivial()
    # and so on for easy, medium, hard
    if task_id == "trivial-doot":
        stack_client.add_trivial()
    elif task_id == "easy-doot":
        stack_client.add_easy()
    elif task_id == "medium-doot":
        stack_client.add_medium()
    elif task_id == "hard-doot":
        stack_client.add_hard()
    else:
        print(f"Unknown task_id: {task_id}")  # Handle unknown task_id gracefully

In [ ]:
monitor = TrelloListMonitor("648a3625f64d43ee787f560f")
notifier = IFTTTNotifier()
habitica = HabiticaAPI(callback=desktop_notifier_callback)

In [ ]:
def handle_trello_changes(diff):
    """Handle all the business logic when Trello changes"""
    if diff['added']:
        for card in diff['added']:
            card_id = card['id']

            card_details = monitor.get_custom_field_items_for_card(card_id)
            print(card)
            card_title = card['name']
            
            if monitor.alter_custom_field_id not in card_details:
                print(card_details)
                print(f"Card '{card_title}' does not have an alter identity set.")
                # we want to set it to a random alter identity of 19, 
                # we can use the alters dictionary we created earlier
                alter_name = monitor.get_random_alter()
                print(f"Setting alter identity to '{alter_name}' for card '{card_title}'")
                monitor.set_custom_field(card_id, monitor.alter_custom_field_id, monitor.alters[alter_name], 'list')
                print(f"Card '{card_title}' has been updated with alter identity '{alter_name}'.")                           


            card_details = monitor.get_card_details(card_id)
            card_title = card_details['title']
            card_story_points = 0.1
            card_frontend_url = card_details.get('frontend_url')

            notifier.send_notification(title=f'Logging Trello Card #{card_title}#...',
                                        message=f'',
                                        url="", image_url="")


            # print(card_title, card_story_points, card_frontend_url)    

            habitica_info = habitica.log_story_points(0.1)
            exp, gp, level = habitica_info['stat_deltas']['exp'], \
                                habitica_info['stat_deltas']['gp'], \
                                habitica_info['stat_deltas']['level']
            
            # we want the title to be dynamic based on exp gp and level, although level might not change
            # emojis used for compression and concision
            # the emoji for exp is the star ⭐
            # the emoji for gold is the coin 🪙
            # the emoji for level is the up arrow ⬆️
            # and we only include the level, gp and exp if they have changed
            title_parts = []
            if exp:
                title_parts.append(f'⭐{round(exp, 2) if isinstance(exp, float) else exp}')
            if gp:
                title_parts.append(f'🪙{round(gp, 2) if isinstance(gp, float) else gp}')
            if level:
                title_parts.append(f'⬆️{level}')

            title = " | ".join(title_parts)

            notifier.send_notification(title='Trello Card In Action! - ' + title,
                                        message=f'specifically this card: ##{card_title}## with story points: {card_story_points}',
                                        url=card_frontend_url)
    if diff['modified']:
        for card in diff['modified']:
            habitica_info = habitica.log_story_points(0.1)



In [ ]:
monitor.monitor(interval=1,callback=handle_trello_changes)